In [ ]:
#install.packages("ggplot2")
#install.packages("openxlsx")
#install.packages("leaflet")
#install.packages("leaflet.extras")
#install.packages("dplyr")
#install.packages("maps")
#install.packages('shiny')
#install.packages('sf')
#install.packages('rvest')
#install.packages("spData")

Libraries

In [3]:
rm(list=ls())
#Useful Libraries
library(ggplot2) 
library(readxl)
library(stats)
library(leaflet)
library(leaflet.extras)
suppressWarnings({library(dplyr)})
library(maps)
library(spData)
library(sf)
library(leaflet.minicharts)
library(rvest)
library(shiny)


### Data Preparation

In this part, the data will be prepared for the analysis:
- Load the data.
- Transform the data.
- Merge datasets for the analysis.


In [4]:
#Preparing the data set
# Set Java heap space to 4GB (adjust as necessary)
#options(java.parameters = "-Xmx8g")
suppressWarnings({
    data=read_excel('D:/Semester 2/Data Analytics and Visualisation/Assignment 1/Twitter+data+in+sheets.xlsx',1) #We pass into R the datasets
    data1=read_excel('D:/Semester 2/Data Analytics and Visualisation/Assignment 1/Twitter+data+in+sheets.xlsx',2) #In this dataset we have the locations for each tweet. data and data1 are linked each other with the variable LocationID
    data2=read_excel('D:/Semester 2/Data Analytics and Visualisation/Assignment 1/Twitter+data+in+sheets.xlsx',3) #In this dataset we have the gender for each observation. data and data2 are linked each other with the variable UserID
    data=data[-100001,]
    k=world.cities 
    dataB=data[seq(50001,nrow(data)),] 
    data=data[-1*seq(50001,nrow(data)),]
})



In [5]:

for (i in seq(1,nrow(data))){
  suppressWarnings({
      data$country[i]=data1$Country[which(data$LocationID[i]==data1$LocationID)]
      data$state[i]=data1$State[which(data$LocationID[i]==data1$LocationID)]
      data$statecode[i]=data1$StateCode[which(data$LocationID[i]==data1$LocationID)]
      data$city[i]=data1$City[which(data$LocationID[i]==data1$LocationID)]
  })
  res = data2$Gender[which(data$UserID[i]==data2$UserID)]
  if (length(res) != 1){
      if ('Unknown' %in% res){
          retur = res[res != 'Unknown'][1]
      } else {
          retur = res[1]
      }
  } else{
      retur = res
  }
  suppressWarnings({data$gender[i] = retur})
}
data$country[data$country=='United States']='USA'
data$country[data$country=='United Kingdom']='UK'
data$country[data$country=='Republic of the Congo']='Congo'



In [6]:
for (i in seq(1,nrow(dataB))){
    suppressWarnings({
      if (length(which(dataB$LocationID[i]==data1$LocationID)!=0)){
        dataB$country[i]=data1$Country[which(dataB$LocationID[i]==data1$LocationID)]
        dataB$state[i]=data1$State[which(dataB$LocationID[i]==data1$LocationID)]
        dataB$statecode[i]=data1$StateCode[which(dataB$LocationID[i]==data1$LocationID)]
        dataB$city[i]=data1$City[which(dataB$LocationID[i]==data1$LocationID)]
      }else{
        dataB$country[i]=0
        dataB$state[i]=0
        dataB$statecode[i]=0
        dataB$city[i]=0
      }
      if(length(which(dataB$UserID[i]==data2$UserID))!=0){
          res = data2$Gender[which(dataB$UserID[i]==data2$UserID)]
          if (length(res) != 1){
              if ('Unknown' %in% res){
                  retur = res[res != 'Unknown'][1]
              } else {
                  retur = res[1]
              }
          } else{
              retur = res
          }
          dataB$gender[i] = retur
      }else{
            dataB$gender[i]=0
          }
        })
}

dataB$country[dataB$country=='United States']='USA'
dataB$country[dataB$country=='United Kingdom']='UK'


In [7]:
data$city[data$statecode=='US-DC']="Washington"
k$country.etc[k$country.etc=='Korea South']='South Korea'
data$city[data$city=='New York City']='New York'
data$city[data$city=="So Paulo"]='Sao Paulo'
data$city[data$city=="stanbul"]='Istanbul'
data$city[data$city=="zmir"]='Izmir'
k$name[k$name=="Roxas" & k$pop>100000]="Roxas City"
k$name[k$name=="Ni Dilli"]="New Delhi"
k$name[k$name=="Milan" & k$pop>1000000]="Milano"
k$name[k$name=='Navi Mumbai']='Mumbai'
k$country.etc[k$name=='Toronto']='USA'


dataB$city[dataB$statecode=='US-DC']="Washington"
dataB$city[dataB$city=='New York City']='New York'
dataB$city[dataB$city=="So Paulo"]='Sao Paulo'
dataB$city[dataB$city=="stanbul"]='Istanbul'
dataB$city[dataB$city=="zmir"]='Izmir'


for(i in seq(1,nrow(data))){
  a=which((data$city[i]==k$name) & (data$country[i]==k$country.etc))
  suppressWarnings({
    if (length(a)!=0){
    data$long[i]=k$long[a][1]
    data$lat[i]=k$lat[a][1]
    data$pop[i]=k$pop[a][1]
  }else{
    data$long[i]=NA
    data$lat[i]=NA
    data$pop[i]=NA
  }
      })
}

for(i in seq(1,nrow(dataB))){
  a=which((dataB$city[i]==k$name) & (dataB$country[i]==k$country.etc))
  suppressWarnings({
    if (length(a)!=0){
    dataB$long[i]=k$long[a][1]
    dataB$lat[i]=k$lat[a][1]
    dataB$pop[i]=k$pop[a][1]
  }else{
    dataB$long[i]=NA
    dataB$lat[i]=NA
    dataB$pop[i]=NA
  }
      })
}


In [8]:
data$long[data$city=="Seattle"]=-122.30
data$lat[data$city=="Seattle"]=47.55
data$long[data$city=="Oldsmar"]=-82.67
data$lat[data$city=="Oldsmar"]=28.05
data$long[data$city=="Ahmedabad"]=72.58
data$lat[data$city=="Ahmedabad"]=23.4
data$long[data$city=="Bogot"]=-74.12
data$lat[data$city=="Bogot"]=4.66
data$long[data$city=="Brooklyn"]=-73.86
data$lat[data$city=="Brooklyn"]=40.68
data$long[data$city=="Boise"]=-116.23
data$lat[data$city=="Boise"]=43.60
data$long[data$city=="Brixton"]=-0.11
data$lat[data$city=="Brixton"]=51.46
data$long[data$city=="Camberley"]=-0.74
data$lat[data$city=="Camberley"]=51.34
data$long[data$city=="Cebu City"]=123.88
data$lat[data$city=="Cebu City"]=10.33
data$long[data$city=="Deeside"]=-3.03
data$lat[data$city=="Deeside"]=53.20
data$long[data$city=="England"]=-91.97
data$lat[data$city=="England"]=34.55
data$long[data$city=="Espaa"]=118.75
data$lat[data$city=="Espaa"]=9.77
data$long[data$city=="Gotham"]=-1.21
data$lat[data$city=="Gotham"]=52.87
data$long[data$city=="Hannover"]=9.75
data$lat[data$city=="Hannover"]=52.37
data$long[data$city=="Herndon"]=-77.38
data$lat[data$city=="Herndon"]=38.97
data$long[data$city=="Kolkata"]=88.37
data$lat[data$city=="Kolkata"]=22.65
data$long[data$city=="Leavenworth"]=-120.66
data$lat[data$city=="Leavenworth"]=47.60
data$long[data$city=="London" & data$state=='California']=-119.44
data$lat[data$city=="London" & data$state=='California']=36.48
data$long[data$city=="Los Gatos"]=-121.97
data$lat[data$city=="Los Gatos"]=37.24
data$long[data$city=="Menlo Park"]=-122.15
data$lat[data$city=="Menlo Park"]=37.48
data$long[data$city=="Oak Grove"]=-87.41
data$lat[data$city=="Oak Grove"]=36.67
data$long[data$city=="Orroli"]=9.25
data$lat[data$city=="Orroli"]=39.70
data$long[data$city=="Ottawa" & data$state=='Ohio']=-84.04
data$lat[data$city=="Ottawa" & data$state=='Ohio']=41.02
data$long[data$city=="Piscataway"]=-74.48
data$lat[data$city=="Piscataway"]=40.54
data$long[data$city=="Regensburg"]=12.10
data$lat[data$city=="Regensburg"]=49.01
data$long[data$city=="Sammamish"]=-122.04
data$lat[data$city=="Sammamish"]=47.62
data$long[data$city=="Scotland"]=-77.59
data$lat[data$city=="Scotland"]=39.97
data$long[data$city=="Shinjuku"]=139.71
data$lat[data$city=="Shinjuku"]=35.70
data$long[data$city=="St. Louis"]=-90.23
data$lat[data$city=="St. Louis"]=38.64
data$long[data$city=="Surrey"]=-122.80
data$lat[data$city=="Surrey"]=49.16
data$long[data$city=="Weil am Rhein"]=7.62
data$lat[data$city=="Weil am Rhein"]=47.59
data$long[data$city=="Woodcliff Lake"]=-74.06
data$lat[data$city=="Woodcliff Lake"]=41.02
data$long[data$city=="Woodside"]=-122.25
data$lat[data$city=="Woodside"]=37.43

data$Weekday=as.factor(data$Weekday)
data$Hour=as.integer(data$Hour)
data$Day=as.integer(data$Day)
data$Lang=as.factor(data$Lang)
data$IsReshare=as.factor(data$IsReshare)
data$Reach=as.integer(data$Reach)
data$RetweetCount=as.integer(data$RetweetCount)
data$Likes=as.integer(data$Likes)
data$Klout=as.integer(data$Klout)
data$Sentiment=as.numeric(data$Sentiment)
data$country=as.factor(data$country)
data$state=as.factor(data$state)
data$city=as.factor(data$city)
data$gender=as.factor(data$gender)
data$UserID=as.factor(data$UserID)
str(data)
summary(data)


dataB$long[dataB$city=="Seattle"]=-122.30
dataB$lat[dataB$city=="Seattle"]=47.55
dataB$long[dataB$city=="Oldsmar"]=-82.67
dataB$lat[dataB$city=="Oldsmar"]=28.05
dataB$long[dataB$city=="Ahmedabad"]=72.58
dataB$lat[dataB$city=="Ahmedabad"]=23.4
dataB$long[dataB$city=="Bogot"]=-74.12
dataB$lat[dataB$city=="Bogot"]=4.66
dataB$long[dataB$city=="Brooklyn"]=-73.86
dataB$lat[dataB$city=="Brooklyn"]=40.68
dataB$long[dataB$city=="Boise"]=-116.23
dataB$lat[dataB$city=="Boise"]=43.60
dataB$long[dataB$city=="Brixton"]=-0.11
dataB$lat[dataB$city=="Brixton"]=51.46
dataB$long[dataB$city=="Camberley"]=-0.74
dataB$lat[dataB$city=="Camberley"]=51.34
dataB$long[dataB$city=="Cebu City"]=123.88
dataB$lat[dataB$city=="Cebu City"]=10.33
dataB$long[dataB$city=="Deeside"]=-3.03
dataB$lat[dataB$city=="Deeside"]=53.20
dataB$long[dataB$city=="England"]=-91.97
dataB$lat[dataB$city=="England"]=34.55
dataB$long[dataB$city=="Espaa"]=118.75
dataB$lat[dataB$city=="Espaa"]=9.77
dataB$long[dataB$city=="Gotham"]=-1.21
dataB$lat[dataB$city=="Gotham"]=52.87
dataB$long[dataB$city=="Hannover"]=9.75
dataB$lat[dataB$city=="Hannover"]=52.37
dataB$long[dataB$city=="Herndon"]=-77.38
dataB$lat[dataB$city=="Herndon"]=38.97
dataB$long[dataB$city=="Kolkata"]=88.37
dataB$lat[dataB$city=="Kolkata"]=22.65
dataB$long[dataB$city=="Leavenworth"]=-120.66
dataB$lat[dataB$city=="Leavenworth"]=47.60
dataB$long[dataB$city=="London" & dataB$state=='California']=-119.44
dataB$lat[dataB$city=="London" & dataB$state=='California']=36.48
dataB$long[dataB$city=="Los Gatos"]=-121.97
dataB$lat[dataB$city=="Los Gatos"]=37.24
dataB$long[dataB$city=="Menlo Park"]=-122.15
dataB$lat[dataB$city=="Menlo Park"]=37.48
dataB$long[dataB$city=="Oak Grove"]=-87.41
dataB$lat[dataB$city=="Oak Grove"]=36.67
dataB$long[dataB$city=="Orroli"]=9.25
dataB$lat[dataB$city=="Orroli"]=39.70
dataB$long[dataB$city=="Ottawa" & dataB$state=='Ohio']=-84.04
dataB$lat[dataB$city=="Ottawa" & dataB$state=='Ohio']=41.02
dataB$long[dataB$city=="Piscataway"]=-74.48
dataB$lat[dataB$city=="Piscataway"]=40.54
dataB$long[dataB$city=="Regensburg"]=12.10
dataB$lat[dataB$city=="Regensburg"]=49.01
dataB$long[dataB$city=="Sammamish"]=-122.04
dataB$lat[dataB$city=="Sammamish"]=47.62
dataB$long[dataB$city=="Scotland"]=-77.59
dataB$lat[dataB$city=="Scotland"]=39.97
dataB$long[dataB$city=="Shinjuku"]=139.71
dataB$lat[dataB$city=="Shinjuku"]=35.70
dataB$long[dataB$city=="St. Louis"]=-90.23
dataB$lat[dataB$city=="St. Louis"]=38.64
dataB$long[dataB$city=="Surrey"]=-122.80
dataB$lat[dataB$city=="Surrey"]=49.16
dataB$long[dataB$city=="Weil am Rhein"]=7.62
dataB$lat[dataB$city=="Weil am Rhein"]=47.59
dataB$long[dataB$city=="Woodcliff Lake"]=-74.06
dataB$lat[dataB$city=="Woodcliff Lake"]=41.02
dataB$long[dataB$city=="Woodside"]=-122.25
dataB$lat[dataB$city=="Woodside"]=37.43

dataB$Weekday=as.factor(dataB$Weekday)
dataB$Hour=as.integer(dataB$Hour)
dataB$Day=as.integer(dataB$Day)
dataB$Lang=as.factor(dataB$Lang)
dataB$IsReshare=as.factor(dataB$IsReshare)
dataB$Reach=as.integer(dataB$Reach)
dataB$RetweetCount=as.integer(dataB$RetweetCount)
dataB$Likes=as.integer(dataB$Likes)
dataB$Klout=as.integer(dataB$Klout)
dataB$Sentiment=as.numeric(dataB$Sentiment)
dataB$country=as.factor(dataB$country)
dataB$state=as.factor(dataB$state)
dataB$city=as.factor(dataB$city)
dataB$gender=as.factor(dataB$gender)
dataB$UserID=as.factor(dataB$UserID)
str(dataB)
summary(dataB)

rm(list=c('k','data1','data2'))


tibble [50,000 × 22] (S3: tbl_df/tbl/data.frame)
 $ TweetID     : chr [1:50000] "tw-682712873332805633" "tw-682713045357998080" "tw-682713219375476736" "tw-682713436967579648" ...
 $ Weekday     : Factor w/ 7 levels "Friday","Monday",..: 5 5 5 5 5 5 5 5 5 5 ...
 $ Hour        : int [1:50000] 17 17 17 17 17 17 17 17 17 17 ...
 $ Day         : int [1:50000] 31 31 31 31 31 31 31 31 31 31 ...
 $ Lang        : Factor w/ 30 levels "_u","ar","cs",..: 7 7 7 7 7 7 7 7 7 7 ...
 $ IsReshare   : Factor w/ 2 levels "FALSE","TRUE": 1 2 1 1 1 1 1 1 1 2 ...
 $ Reach       : int [1:50000] 44 1810 282 2087 953 113 113 113 4893 2868 ...
 $ RetweetCount: int [1:50000] 0 5 0 4 0 0 0 0 1 6 ...
 $ Likes       : int [1:50000] 0 0 0 0 0 0 0 0 0 0 ...
 $ Klout       : int [1:50000] 35 53 47 53 47 31 31 31 49 51 ...
 $ Sentiment   : num [1:50000] 0 2 0 0 0 -1 -2 -1 0 2 ...
 $ text        : chr [1:50000] "We are hiring: Senior Software Engineer - Proto http://www.reqcloud.com/jobs/719865/?k=0LaPxXuFwczs1e32ZURJKr

   TweetID               Weekday          Hour            Day       
 Length:50000       Friday   :8051   Min.   : 0.00   Min.   : 1.00  
 Class :character   Monday   :7454   1st Qu.: 7.00   1st Qu.: 8.00  
 Mode  :character   Saturday :3666   Median :11.00   Median :13.00  
                    Sunday   :3487   Mean   :11.49   Mean   :13.82  
                    Thursday :8461   3rd Qu.:16.00   3rd Qu.:20.00  
                    Tuesday  :9842   Max.   :23.00   Max.   :31.00  
                    Wednesday:9039                                  
      Lang       IsReshare         Reach          RetweetCount   
 en     :47892   FALSE:32398   Min.   :      0   Min.   :  0.00  
 _u     :  476   TRUE :17602   1st Qu.:    153   1st Qu.:  0.00  
 es     :  435                 Median :    430   Median :  0.00  
 fr     :  328                 Mean   :   7336   Mean   :  8.39  
 ja     :  150                 3rd Qu.:   1352   3rd Qu.:  3.00  
 de     :  130                 Max.   :8196371   Max

tibble [50,000 × 22] (S3: tbl_df/tbl/data.frame)
 $ TweetID     : chr [1:50000] "tw-699956523758387201" "tw-699956772468105217" "tw-699956793636753408" "tw-699956949002149888" ...
 $ Weekday     : Factor w/ 7 levels "Friday","Monday",..: 7 7 7 7 7 7 7 7 7 7 ...
 $ Hour        : int [1:50000] 7 7 7 7 7 7 7 7 7 7 ...
 $ Day         : int [1:50000] 17 17 17 17 17 17 17 17 17 17 ...
 $ Lang        : Factor w/ 35 levels "_u","ar","cs",..: 7 7 7 23 7 12 7 7 33 7 ...
 $ IsReshare   : Factor w/ 2 levels "FALSE","TRUE": 2 1 2 1 2 1 1 1 1 1 ...
 $ Reach       : int [1:50000] 2095 4540 11 258 134 150 780 107 42060 52 ...
 $ RetweetCount: int [1:50000] 1 1 1 4 2 0 0 0 0 0 ...
 $ Likes       : int [1:50000] 0 0 0 0 0 0 0 0 0 0 ...
 $ Klout       : int [1:50000] 55 52 14 31 27 27 57 29 50 41 ...
 $ Sentiment   : num [1:50000] 0 0 0 0 0 0 2 0 0 0 ...
 $ text        : chr [1:50000] "RT @mchamberlain99: #Dropbox expanding into #Europe hosting customer #data in #Germany http://www.eweek.com/sto"| __trun

   TweetID               Weekday          Hour            Day       
 Length:50000       Friday   :6953   Min.   : 0.00   Min.   : 1.00  
 Class :character   Monday   :7966   1st Qu.: 7.00   1st Qu.:13.00  
 Mode  :character   Saturday :3755   Median :11.00   Median :19.00  
                    Sunday   :3506   Mean   :11.33   Mean   :17.97  
                    Thursday :9997   3rd Qu.:16.00   3rd Qu.:24.00  
                    Tuesday  :8626   Max.   :23.00   Max.   :31.00  
                    Wednesday:9197                                  
      Lang       IsReshare         Reach           RetweetCount      
 en     :43994   FALSE:32140   Min.   :       0   Min.   :    0.000  
 es     : 1265   TRUE :17860   1st Qu.:     148   1st Qu.:    0.000  
 tr     :  847                 Median :     470   Median :    0.000  
 fr     :  757                 Mean   :    9749   Mean   :    7.715  
 ja     :  521                 3rd Qu.:    1673   3rd Qu.:    3.000  
 de     :  433              

### Data Analysis (Part 1 / 4)

In this part we will conduct data analysis:
- Number of tweets in each country.
- Mean, min, max, median for the Sentiment variable for each country.
- Number of tweets for each gender in each country.
- Create Map with polygons (statistics for each country).

In [9]:

set=data %>% group_by(country) %>% count()
set1=data %>% group_by(country) %>% summarise(mean=mean(Sentiment),max=max(Sentiment),min=min(Sentiment),median=median(Sentiment),meanR=mean(Reach))
set2=data %>% group_by(country,gender) %>% count()
View(set2)

setB=dataB %>% group_by(country) %>% count()
set1B=dataB %>% group_by(country) %>% summarise(mean=mean(Sentiment),max=max(Sentiment),min=min(Sentiment),median=median(Sentiment),meanR=mean(Reach))
set2B=dataB %>% group_by(country,gender) %>% count()
View(set1)
View(set)
str(set2)

View(set1B)
View(setB)
polyg=world 
polyg$name_long[polyg$name_long=="United States"]="USA"
polyg$name_long[polyg$name_long=="United Kingdom"]="UK"
polyg$name_long[polyg$name_long=='Republic of Korea']="South Korea"
polyg$name_long[polyg$name_long=='Dem. Rep. Korea']="North Korea"
polyg$name_long[polyg$name_long=="Côte d'Ivoire"]="Ivory Coast"
polyg$name_long[polyg$name_long=="Republic of the Congo"]="Congo"
polyg$numTwe=NA
polyg$max=NA
polyg$min=NA
polyg$mean=NA
polyg$median=NA
polyg$meanR=NA
for ( i in seq(1,nrow(polyg))){
  if (length(which(set$country==polyg$name_long[i]))!=0){  
    polyg$numTwe[i]=set$n[which(set$country==polyg$name_long[i])]
    polyg$max[i]=set1$max[which(set1$country==polyg$name_long[i])]
    polyg$min[i]=set1$min[which(set1$country==polyg$name_long[i])]
    polyg$mean[i]=set1$mean[which(set1$country==polyg$name_long[i])]
    polyg$median[i]=set1$median[which(set1$country==polyg$name_long[i])]
    polyg$meanR[i]=set1$meanR[which(set1$country==polyg$name_long[i])]
  }
}
polygB=world
polygB$name_long[polygB$name_long=="United States"]="USA"
polygB$name_long[polygB$name_long=="United Kingdom"]="UK"
polygB$name_long[polygB$name_long=='Republic of Korea']="South Korea"
polygB$name_long[polygB$name_long=='Dem. Rep. Korea']="North Korea"
polygB$name_long[polygB$name_long=="Côte d'Ivoire"]="Ivory Coast"
polygB$name_long[polygB$name_long=="Republic of the Congo"]="Congo"
polygB$numTwe=NA
polygB$max=NA
polygB$min=NA
polygB$mean=NA
polygB$median=NA
polygB$meanR=NA
for ( i in seq(1,nrow(polygB))){
  if (length(which(setB$country==polygB$name_long[i]))!=0){  
    polygB$numTwe[i]=setB$n[which(setB$country==polygB$name_long[i])]
    polygB$max[i]=set1B$max[which(set1B$country==polygB$name_long[i])]
    polygB$min[i]=set1B$min[which(set1B$country==polygB$name_long[i])]
    polygB$mean[i]=set1B$mean[which(set1B$country==polygB$name_long[i])]
    polygB$median[i]=set1B$median[which(set1B$country==polygB$name_long[i])]
    polygB$meanR[i]=set1B$meanR[which(set1B$country==polygB$name_long[i])]
  }
}
colcont=colorNumeric(palette = "Blues",domain =log(c(1:26426)))
colcontB=colorNumeric(palette = "Blues",domain =log(c(min(polygB$numTwe,na.rm=T):max(polygB$numTwe,na.rm=T))))#We create a continuous palette for the second dataset polygB

coloring=colorFactor(palette = c("Red","Blue","Black","Green"),levels = c("Female","Male","Unisex","Unknown"))
labelmap=function(x,y,z,w,e){paste0("Sentiment","<br/>","Minimum:",x,"<br/>","Maximum:",y,"<br/>","Mean:",z,"<br/>","Median:",w,"<br/>",'Mean Reach:',e )}

map=leaflet() %>% 
  addProviderTiles("CartoDB") %>% 
  addPolygons(data = polyg,
              weight = 1,
              fillOpacity = 0.75,
              color= ~colcont(log(numTwe)),
              label = paste('Number of Tweets',polyg$numTwe),
              popup = ~labelmap(min,max,mean,median,meanR),
              highlight=highlightOptions(weight = 3,color = "green")) %>%
  addLegend(pal = colcont,
            values = log(polyg$numTwe),
            title = "#twts = e^lgnd")
mapB=leaflet() %>%
  addProviderTiles("CartoDB") %>% 
  addPolygons(data = polygB,
              weight = 1,
              fillOpacity = 0.75,
              color= ~colcontB(log(numTwe)),
              label = paste('Number of Tweets',polygB$numTwe),
              popup = ~labelmap(min,max,mean,median,meanR),
              highlight=highlightOptions(weight = 3,color = "green")) %>%
  addLegend(pal = colcontB,
            values = log(polygB$numTwe),
            title = "#twts = e^lgnd")


country,gender,n
<fct>,<fct>,<int>
Albania,Male,4
Albania,Unknown,1
Algeria,Male,3
Angola,Female,5
Anguilla,Male,1
Argentina,Female,49
Argentina,Male,110
Argentina,Unisex,29
Argentina,Unknown,60


country,mean,max,min,median,meanR
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Albania,0.0000000,0,0.0,0.0,137.00000
Algeria,-1.0000000,0,-3.0,0.0,180.66667
Angola,-1.2000000,0,-2.0,-1.0,67.40000
Anguilla,0.0000000,0,0.0,0.0,267.00000
Argentina,0.2336102,4,-4.0,0.0,1136.25806
Armenia,0.0000000,0,0.0,0.0,98.50000
Australia,0.4143995,5,-6.0,0.0,1694.62359
Austria,0.4643672,4,-4.0,0.0,666.67949
Bahamas,0.0000000,0,0.0,0.0,141.50000


country,n
<fct>,<int>
Albania,5
Algeria,3
Angola,5
Anguilla,1
Argentina,248
Armenia,2
Australia,1416
Austria,78
Bahamas,10


gropd_df [370 × 3] (S3: grouped_df/tbl_df/tbl/data.frame)
 $ country: Factor w/ 136 levels "Albania","Algeria",..: 1 1 2 3 4 5 5 5 5 6 ...
 $ gender : Factor w/ 4 levels "Female","Male",..: 2 4 2 1 2 1 2 3 4 1 ...
 $ n      : int [1:370] 4 1 3 5 1 49 110 29 60 1 ...
 - attr(*, "groups")= tibble [370 × 3] (S3: tbl_df/tbl/data.frame)
  ..$ country: Factor w/ 136 levels "Albania","Algeria",..: 1 1 2 3 4 5 5 5 5 6 ...
  ..$ gender : Factor w/ 4 levels "Female","Male",..: 2 4 2 1 2 1 2 3 4 1 ...
  ..$ .rows  : list<int> [1:370] 
  .. ..$ : int 1
  .. ..$ : int 2
  .. ..$ : int 3
  .. ..$ : int 4
  .. ..$ : int 5
  .. ..$ : int 6
  .. ..$ : int 7
  .. ..$ : int 8
  .. ..$ : int 9
  .. ..$ : int 10
  .. ..$ : int 11
  .. ..$ : int 12
  .. ..$ : int 13
  .. ..$ : int 14
  .. ..$ : int 15
  .. ..$ : int 16
  .. ..$ : int 17
  .. ..$ : int 18
  .. ..$ : int 19
  .. ..$ : int 20
  .. ..$ : int 21
  .. ..$ : int 22
  .. ..$ : int 23
  .. ..$ : int 24
  .. ..$ : int 25
  .. ..$ : int 26
  .. ..$ : 

country,mean,max,min,median,meanR
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Albania,1.00000000,2.000000,0.00000000,1.00,86.0000
Anguilla,2.00000000,2.000000,2.00000000,2.00,239.0000
Argentina,0.18869056,3.000000,-2.33333325,0.00,1306.9639
Australia,0.37932134,4.500000,-4.00000000,0.00,1595.0920
Austria,0.28212766,3.000000,-2.50000000,0.00,1115.1489
Bangladesh,0.41904762,4.000000,-6.00000000,0.00,1545.6000
Belarus,0.16636996,1.000000,-0.01310044,0.00,496.7500
Belgium,0.65188889,4.000000,-3.00000000,0.00,1946.9111
Bolivia,0.12461538,2.000000,-1.00000000,0.00,958.2308


country,n
<fct>,<int>
Albania,2
Anguilla,1
Argentina,277
Australia,1521
Austria,94
Bangladesh,105
Belarus,8
Belgium,90
Bolivia,13


### Data Analysis (Part 2 / 4)

In this part we will create a map showing the exact location that the tweet was made by a user along with some info about the tweet and the user, such as, their gender, number of retweets, the language of the tweet and the Klout score.

In [10]:
for (i in unique(data$UserID)){#2-3mins
  while (TRUE){
    run=runif(1,-0.01,0.01)
    run1=runif(1,-0.01,0.01)
    if (run!=0 & run1!=0){
      break
    }
  }
  data$long[data$UserID==i]=data$long[data$UserID==i]+run 
  data$lat[data$UserID==i]=data$lat[data$UserID==i]+run1
}

for (i in unique(dataB$UserID)){#2-3 mins
  while (TRUE){
    run=runif(1,-0.01,0.01)
    run1=runif(1,-0.01,0.01)
    if (run!=0 & run1!=0){
      break
    }
  }
  dataB$long[dataB$UserID==i]=dataB$long[dataB$UserID==i]+run 
  dataB$lat[dataB$UserID==i]=dataB$lat[dataB$UserID==i]+run1
}


popup=function(x,y) {paste0("Language: ",x,"<br/>","Number of Retweets: ",y)}
label= function(x) {paste("Klout: ",x)}
male=data %>% filter(gender=="Male")
female=data %>% filter(gender=="Female")
unisex=data %>% filter(gender=="Unisex")
unknown=data %>% filter(gender=="Unknown")

suppressWarnings({map1=leaflet() %>%
  addProviderTiles('CartoDB') %>%
  addCircleMarkers(data=male,
                   lng = ~long,
                   lat = ~lat,
                   radius = 1.4,
                   clusterOptions = markerClusterOptions(),
                   label = ~label(Klout),
                   popup = ~popup(Lang,RetweetCount),
                   color = ~coloring(gender),
                   group = "Male"
                   ) %>%
  addCircleMarkers(data=female,
                   lng = ~long,
                   lat = ~lat,
                   radius = 1.4,
                   clusterOptions = markerClusterOptions(),
                   label = ~label(Klout),
                   popup = ~popup(Lang,RetweetCount),
                   color = ~coloring(gender),
                   group = "Female" 
                   ) %>%
  addCircleMarkers(data=unisex,
                   lng = ~long,
                   lat = ~lat,
                   radius = 1.4,
                   clusterOptions = markerClusterOptions(),
                   label = ~label(Klout),
                   popup = ~popup(Lang,RetweetCount),
                   color = ~coloring(gender),
                   group = "Unisex" 
                   ) %>%
  addCircleMarkers(data=unknown,
                   lng = ~long,
                   lat = ~lat,
                   radius = 1.4,
                   clusterOptions = markerClusterOptions(),
                   label = ~label(Klout),
                   popup = ~popup(Lang,RetweetCount),
                   color = ~coloring(gender),
                   group = "Unknown" 
                   ) %>%
  addLegend(pal = coloring,
            values = c("Female","Male","Unisex","Unknown")) %>%
  addLayersControl(overlayGroups = c("Male","Female",'Unisex','Unknown'))
})  

maleB=dataB %>% filter(gender=="Male")
femaleB=dataB %>% filter(gender=="Female")
unisexB=dataB %>% filter(gender=="Unisex")
unknownB=dataB %>% filter(gender=="Unknown")
suppressWarnings({map1B=leaflet() %>%
  addProviderTiles('CartoDB') %>%
  addCircleMarkers(data=maleB,
                   lng = ~long,
                   lat = ~lat,
                   radius = 1.4,
                   clusterOptions = markerClusterOptions(),
                   label = ~label(Klout),
                   popup = ~popup(Lang,RetweetCount),
                   color = ~coloring(gender),
                   group = "Male"
  ) %>%
  addCircleMarkers(data=femaleB,
                   lng = ~long,
                   lat = ~lat,
                   radius = 1.4,
                   clusterOptions = markerClusterOptions(),
                   label = ~label(Klout),
                   popup = ~popup(Lang,RetweetCount),
                   color = ~coloring(gender),
                   group = "Female" 
  ) %>%
  addCircleMarkers(data=unisexB,
                   lng = ~long,
                   lat = ~lat,
                   radius = 1.4,
                   clusterOptions = markerClusterOptions(),
                   label = ~label(Klout),
                   popup = ~popup(Lang,RetweetCount),
                   color = ~coloring(gender),
                   group = "Unisex" 
  ) %>%
  addCircleMarkers(data=unknownB,
                   lng = ~long,
                   lat = ~lat,
                   radius = 1.4,
                   clusterOptions = markerClusterOptions(),
                   label = ~label(Klout),
                   popup = ~popup(Lang,RetweetCount),
                   color = ~coloring(gender),
                   group = "Unknown" 
  ) %>%
  addLegend(pal = coloring,
            values = c("Female","Male","Unisex","Unknown")) %>%
  addLayersControl(overlayGroups = c("Male","Female",'Unisex','Unknown')) 
})


Web Scraping to write countries dataset.

In [ ]:
#Web scraping
#URL="https://developers.google.com/public-data/docs/canonical/countries_csv"

#web=read_html(URL)

#all=web %>%
#  html_nodes("td") %>% html_text()

#df=data.frame(code=all[seq(1,length(all),4)],latitude=all[seq(2,length(all),4)],longtitude=all[seq(3,length(all),4)],country=all[seq(4,length(all),4)])

#write.csv(df,'D:/Semester 2/Data Analytics and Visualisation/Assignment 1/countries.csv')

In [11]:
countries=read.csv("D:/Semester 2/Data Analytics and Visualisation/Assignment 1/countries.csv")
countries$country[countries$country=="United States"]="USA" 
countries$country[countries$country=="United Kingdom"]="UK"
countries$country[countries$code=="CI"]="Ivory Coast"
countries$country[countries$country=="Congo [DRC]"]="Democratic Republic of the Congo"
countries$country[countries$country=="Congo [Republic]"]="Congo"
str(countries)
View(countries)

for (i in unique(set2$country)){
  if(length(which(countries$country==i))==0){next}else{
    x=set2$n[(set2$country==i) & (set2$gender=="Male")]
    y=set2$n[(set2$country==i) & (set2$gender=="Female")]
    z=set2$n[(set2$country==i) & (set2$gender=="Unisex")]
    w=set2$n[(set2$country==i) & (set2$gender=="Unknown")]
    if (length(x)==0){x=0}
    if (length(y)==0){y=0}
    if (length(z)==0){z=0}
    if (length(w)==0){w=0}
    map=map %>%
      addMinicharts(lng = countries$longtitude[countries$country==i],
                    lat = countries$latitude[countries$country==i],
                    chartdata =c(x,y,z,w),
                    type = 'bar',
                    opacity = 1,
                    width = 22,
                    height = 32)
  }
}

map=map %>% addLegend(position = 'bottomright',
                      pal = colorFactor(palette = c("blue",'orange','green','red'),
                                        levels = c("Male","Female","Unisex","Unknown")),
                      values = c("Male","Female","Unisex","Unknown"),
                      title = "Bar Colors")



'data.frame':	245 obs. of  5 variables:
 $ X         : int  1 2 3 4 5 6 7 8 9 10 ...
 $ code      : chr  "AD" "AE" "AF" "AG" ...
 $ latitude  : num  42.5 23.4 33.9 17.1 18.2 ...
 $ longtitude: num  1.6 53.8 67.7 -61.8 -63.1 ...
 $ country   : chr  "Andorra" "United Arab Emirates" "Afghanistan" "Antigua and Barbuda" ...


X,code,latitude,longtitude,country
<int>,<chr>,<dbl>,<dbl>,<chr>
1,AD,42.546245,1.601554,Andorra
2,AE,23.424076,53.847818,United Arab Emirates
3,AF,33.939110,67.709953,Afghanistan
4,AG,17.060816,-61.796428,Antigua and Barbuda
5,AI,18.220554,-63.068615,Anguilla
6,AL,41.153332,20.168331,Albania
7,AM,40.069099,45.038189,Armenia
8,AN,12.226079,-69.060087,Netherlands Antilles
9,AO,-11.202692,17.873887,Angola


### Data Analysis (Part 3 / 4)

In this part we perform multivariate analysis displaying scatter plots of Reach vs Klout. Each point of the scatter plot is a tweets and for each of them we can get relevant info such as, Gender, whether has been reshared or not, number of likes, and sentiment. Different transformations can also be applied (Arctan and Log) for visualisation reasons. 

In [12]:

for (i in unique(set2B$country)){
  if(length(which(countries$country==i))==0){next}else{
    x=set2B$n[(set2B$country==i) & (set2B$gender=="Male")]
    y=set2B$n[(set2B$country==i) & (set2B$gender=="Female")]
    z=set2B$n[(set2B$country==i) & (set2B$gender=="Unisex")]
    w=set2B$n[(set2B$country==i) & (set2B$gender=="Unknown")]
    if (length(x)==0){x=0}
    if (length(y)==0){y=0}
    if (length(z)==0){z=0}
    if (length(w)==0){w=0}
    mapB=mapB %>%
      addMinicharts(lng = countries$longtitude[countries$country==i],
                    lat = countries$latitude[countries$country==i],
                    chartdata =c(x,y,z,w),
                    type = 'bar',
                    opacity = 1,
                    width = 22,
                    height = 32)
  }
}
mapB=mapB %>% addLegend(position = 'bottomright',
                        pal = colorFactor(palette = c("blue",'orange','green','red'),
                                          levels = c("Male","Female","Unisex","Unknown")),
                        values = c("Male","Female","Unisex","Unknown"),
                        title = "Bar Colors")







dfA=data
dfB=dataB
str(dfA)
str(dfB)
View(dfB)

dfA$Sentiment[dfA$Sentiment>1]="Much Positive"
dfA$Sentiment[dfA$Sentiment>0 & dfA$Sentiment<=1]="Positive"
dfA$Sentiment[dfA$Sentiment==0]="Neutral"
dfA$Sentiment[dfA$Sentiment>=-1 & dfA$Sentiment<0]="Negative"
dfA$Sentiment[dfA$Sentiment< -1]="Much Negative"

dfB$Sentiment[dfB$Sentiment>1]="Much Positive"
dfB$Sentiment[dfB$Sentiment>0 & dfB$Sentiment<=1]="Positive"
dfB$Sentiment[dfB$Sentiment==0]="Neutral"
dfB$Sentiment[dfB$Sentiment>=-1 & dfB$Sentiment<0]="Negative"
dfB$Sentiment[dfB$Sentiment< -1]="Much Negative"
dfB$gender[dfB$gender==0]="Unknown"
View(dataB)

dframe=data[,-c(12:16)]


tibble [50,000 × 22] (S3: tbl_df/tbl/data.frame)
 $ TweetID     : chr [1:50000] "tw-682712873332805633" "tw-682713045357998080" "tw-682713219375476736" "tw-682713436967579648" ...
 $ Weekday     : Factor w/ 7 levels "Friday","Monday",..: 5 5 5 5 5 5 5 5 5 5 ...
 $ Hour        : int [1:50000] 17 17 17 17 17 17 17 17 17 17 ...
 $ Day         : int [1:50000] 31 31 31 31 31 31 31 31 31 31 ...
 $ Lang        : Factor w/ 30 levels "_u","ar","cs",..: 7 7 7 7 7 7 7 7 7 7 ...
 $ IsReshare   : Factor w/ 2 levels "FALSE","TRUE": 1 2 1 1 1 1 1 1 1 2 ...
 $ Reach       : int [1:50000] 44 1810 282 2087 953 113 113 113 4893 2868 ...
 $ RetweetCount: int [1:50000] 0 5 0 4 0 0 0 0 1 6 ...
 $ Likes       : int [1:50000] 0 0 0 0 0 0 0 0 0 0 ...
 $ Klout       : int [1:50000] 35 53 47 53 47 31 31 31 49 51 ...
 $ Sentiment   : num [1:50000] 0 2 0 0 0 -1 -2 -1 0 2 ...
 $ text        : chr [1:50000] "We are hiring: Senior Software Engineer - Proto http://www.reqcloud.com/jobs/719865/?k=0LaPxXuFwczs1e32ZURJKr

TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,⋯,LocationID,UserID,country,state,statecode,city,gender,long,lat,pop
<chr>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>,<int>,⋯,<dbl>,<fct>,<fct>,<fct>,<chr>,<fct>,<fct>,<dbl>,<dbl>,<int>
tw-699956523758387201,Wednesday,7,17,en,TRUE,2095,1,0,55,⋯,3048,tw-135150713,UK,County of Flintshire,GB-Y1,Deeside,Male,-3.03128993,53.20506,NA
tw-699956772468105217,Wednesday,7,17,en,FALSE,4540,1,0,52,⋯,3019,tw-52748391,UK,Greater London,GB-GLA,London,Unknown,-0.10297678,51.51037,7489022
tw-699956793636753408,Wednesday,7,17,en,TRUE,11,1,0,14,⋯,3851,tw-3145304566,USA,California,US-CA,Sacramento,Male,-121.47164743,38.57758,480392
tw-699956949002149888,Wednesday,7,17,nl,FALSE,258,4,0,31,⋯,2055,tw-238048977,Netherlands,North Holland,NL,Amsterdam,Male,4.88597356,52.36615,744159
tw-699957060495155200,Wednesday,7,17,en,TRUE,134,2,0,27,⋯,3927,tw-24172457,USA,Minnesota,US-MN,Minneapolis,Female,-93.26520052,44.96823,364726
tw-699957889637593089,Wednesday,7,17,fr,FALSE,150,0,0,27,⋯,800,tw-2343826152,France,Ile-de-France,FR,Paris,Female,2.33214259,48.85231,2141839
tw-699957890317225985,Wednesday,7,17,en,FALSE,780,0,0,57,⋯,3745,tw-2588391314,USA,Nevada,US-NV,Reno,Male,-119.81365833,39.53853,206626
tw-699957909350965248,Wednesday,7,17,en,FALSE,107,0,0,29,⋯,712,tw-24527668,Denmark,Capital Region,DK,Copenhagen,Male,12.56317261,55.67525,1091978
tw-699958989329690624,Wednesday,7,17,tr,FALSE,42060,0,0,50,⋯,2963,tw-2903118737,Turkey,Istanbul,TR,Istanbul,Unknown,29.00968050,41.10621,10034830


TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,⋯,LocationID,UserID,country,state,statecode,city,gender,long,lat,pop
<chr>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>,<int>,⋯,<dbl>,<fct>,<fct>,<fct>,<chr>,<fct>,<fct>,<dbl>,<dbl>,<int>
tw-699956523758387201,Wednesday,7,17,en,TRUE,2095,1,0,55,⋯,3048,tw-135150713,UK,County of Flintshire,GB-Y1,Deeside,Male,-3.03128993,53.20506,NA
tw-699956772468105217,Wednesday,7,17,en,FALSE,4540,1,0,52,⋯,3019,tw-52748391,UK,Greater London,GB-GLA,London,Unknown,-0.10297678,51.51037,7489022
tw-699956793636753408,Wednesday,7,17,en,TRUE,11,1,0,14,⋯,3851,tw-3145304566,USA,California,US-CA,Sacramento,Male,-121.47164743,38.57758,480392
tw-699956949002149888,Wednesday,7,17,nl,FALSE,258,4,0,31,⋯,2055,tw-238048977,Netherlands,North Holland,NL,Amsterdam,Male,4.88597356,52.36615,744159
tw-699957060495155200,Wednesday,7,17,en,TRUE,134,2,0,27,⋯,3927,tw-24172457,USA,Minnesota,US-MN,Minneapolis,Female,-93.26520052,44.96823,364726
tw-699957889637593089,Wednesday,7,17,fr,FALSE,150,0,0,27,⋯,800,tw-2343826152,France,Ile-de-France,FR,Paris,Female,2.33214259,48.85231,2141839
tw-699957890317225985,Wednesday,7,17,en,FALSE,780,0,0,57,⋯,3745,tw-2588391314,USA,Nevada,US-NV,Reno,Male,-119.81365833,39.53853,206626
tw-699957909350965248,Wednesday,7,17,en,FALSE,107,0,0,29,⋯,712,tw-24527668,Denmark,Capital Region,DK,Copenhagen,Male,12.56317261,55.67525,1091978
tw-699958989329690624,Wednesday,7,17,tr,FALSE,42060,0,0,50,⋯,2963,tw-2903118737,Turkey,Istanbul,TR,Istanbul,Unknown,29.00968050,41.10621,10034830


### Data Analysis (Part 4 / 4)

In this part we investigate each variable for each gender depending on time.

In [14]:
date=c()
j=1
for (i in seq(1:nrow(dframe))){
  if (dframe$Day[i]==dframe$Day[i+1]){
    date=c(date,j)
  }else{
    date=c(date,j)
    j=j+1
  }
  if (i==49999){
    break
  }
}
tail(dframe)
date=c(date,j)
dframe$time=date
suppressWarnings({dframeA= dframe %>%
  group_by(time,gender) %>% 
  summarise(Reach=mean(Reach),
            sentiment=mean(Sentiment),
            retw=mean(RetweetCount),
            likes=mean(Likes),
            klout=mean(Klout),.groups = "keep")
})


dfra= dframe %>%
  group_by(time,gender) %>%
  count()

dframeA$count=dfra$n

males=dframeA %>% filter(gender=="Male")
females=dframeA %>% filter(gender=="Female")
unknowns=dframeA %>% filter(gender=="Unknown")
unisexs=dframeA %>% filter(gender=="Unisex")

dframe1=dataB[,-c(12:16)]
date=c()
j=1
for (i in seq(1:nrow(dframe1))){
  if (dframe1$Day[i]==dframe1$Day[i+1]){
    date=c(date,j)
  }else{
    date=c(date,j)
    j=j+1
  }
  if (i==49999){
    break
  }
}
date=c(date,j)
dframe1$time=date
dframe1A= dframe1 %>%
  group_by(time,gender) %>% 
  summarise(Reach=mean(Reach),
            sentiment=mean(Sentiment),
            retw=mean(RetweetCount),
            likes=mean(Likes),
            klout=mean(Klout),.groups = "keep")



dfra1= dframe1 %>%
  group_by(time,gender) %>%
  count()

dframe1A$count=dfra1$n

males1=dframe1A %>% filter(gender=="Male")
females1=dframe1A %>% filter(gender=="Female")
unknowns1=dframe1A %>% filter(gender=="Unknown")
unisexs1=dframe1A %>% filter(gender=="Unisex")



TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,statecode,city,gender,long,lat,pop,time
<chr>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<fct>,<fct>,<dbl>,<dbl>,<int>,<dbl>
tw-699955608447901697,Wednesday,6,17,en,TRUE,223,27,0,30,0.0,US-DC,Washington,Male,-77.029843,38.91163,548359,49
tw-699955839315111937,Wednesday,6,17,en,FALSE,1180,0,0,51,0.0,US-CA,Santa Monica,Unknown,-118.482260,34.01176,89112,49
tw-699955903060021248,Wednesday,6,17,en,TRUE,17894,1,0,48,0.0,US-CO,Westminster,Unknown,-105.044282,39.88114,104708,49
tw-699956028054642689,Wednesday,6,17,en,FALSE,289,0,0,27,1.5,US-WA,Seattle,Male,-122.303523,47.54527,570430,49
tw-699956462492094464,Wednesday,6,17,en,TRUE,1229,4,0,50,3.0,US-CA,Mountain View,Female,-122.073564,37.39174,67788,49
tw-699956510387003393,Wednesday,7,17,en,TRUE,265,3,0,39,3.0,FR,Paris,Male,2.341617,48.86428,2141839,49


### Shiny App

In this part we create an interactive app to display the analysis.
The application has 4 tabs:
1) The Map with the exact points of the tweets.
2) The Map with the polygons and the statistics for each country.
3) The Multivariate analysis with the scatter plots.
4) The Timeseries.

In [ ]:
#Shiny App
ui=fluidPage(navlistPanel(tabPanel('Point Map',
                                   leafletOutput(outputId = 'map1'),
                                   leafletOutput(outputId = 'map1B')),
                          tabPanel("Polygons Map",
                                   leafletOutput(outputId = 'map'),
                                   leafletOutput('mapB')),
                          tabPanel("Graphs",
                                   sidebarLayout(sidebarPanel(selectInput(inputId = 'facet1',
                                                                          'Select a variable to split the scatter plot',
                                                                          choices = c('gender','IsReshare','No')),
                                                               selectInput(inputId = 'facet2',
                                                                           'Select a second variable to split the scatter plot',
                                                                           choices = c('gender','IsReshare','No')),
                                                               selectInput(inputId = 'size1',
                                                                           'Select a variable for size',
                                                                            choices = c("Likes", "RetweetCount",NULL)),
                                                              selectInput(inputId = 'Fun',
                                                                          "Select transformation",
                                                                          choices = c('Logarithm',"Arctan","Nothing"))
                                                               ),
                                                 mainPanel(plotOutput(outputId = 'plt1'),
                                                           plotOutput(outputId = 'plt2')
                                                           )
                                                 )
                                   ),
                          tabPanel("Timeseries",
                                   sidebarLayout(sidebarPanel(selectInput(inputId = 'time',
                                                                          'Select variable for timeseries',
                                                                          choices = names(dframeA)[3:8]
                                                                          )),
                                                 mainPanel(plotOutput(outputId = 'plt3'),
                                                           plotOutput(outputId = 'plt4'))
                                                 )
                                   )
                          )
             ) 


server=function(input,output){
  transf=reactive({if (input$Fun=='Logarithm'){
    'log(Reach)'
  }else if (input$Fun=='Arctan'){
    'atan(log(Reach)-6.29)'
  }else {"Reach"}
 })
  plt=reactive({ggplot(data = dfA,aes_string(x = transf(), y = "Klout", color = "Sentiment", size = input$size1))+  
      geom_point()+
      scale_colour_discrete(type = c('red','green','yellow','pink','blue'))+
      theme_dark()})
  plt1=reactive({ggplot(data = dfB,aes_string(x = transf(), y = "Klout", color = "Sentiment", size = input$size1))+  
      geom_point()+
      scale_colour_discrete(type = c('red','green','yellow','pink','blue'))+
      theme_dark()})
  output$map1=renderLeaflet(map1)
  output$map1B=renderLeaflet(map1B)
  output$map=renderLeaflet(map)
  output$mapB=renderLeaflet(mapB)
  output$plt1=renderPlot({if (input$facet1=='No' & input$facet1=='No'){
    plt()}else if ((input$facet1=="No" & input$facet2=='gender') | (input$facet1=='gender' & input$facet2=='gender') | (input$facet2=="No" & input$facet1=='gender')){
    plt()+
      facet_wrap(~gender)
      }else if((input$facet1=="No" & input$facet2=='IsReshare') | (input$facet1=='IsReshare' & input$facet2=='IsReshare') | (input$facet2=="No" & input$facet1=='IsReshare')){
        plt()+
          facet_wrap(~IsReshare)
      }else{plt()+
             facet_wrap(~IsReshare+gender)}})
  output$plt2=renderPlot({if (input$facet1=='No' & input$facet1=='No'){
    plt1()}else if ((input$facet1=="No" & input$facet2=='gender') | (input$facet1=='gender' & input$facet2=='gender') | (input$facet2=="No" & input$facet1=='gender')){
      plt1()+
        facet_wrap(~gender)
    }else if((input$facet1=="No" & input$facet2=='IsReshare') | (input$facet1=='IsReshare' & input$facet2=='IsReshare') | (input$facet2=="No" & input$facet1=='IsReshare')){
      plt1()+
        facet_wrap(~IsReshare)
    }else{plt1()+
             facet_wrap(~IsReshare+gender)}})
  output$plt3=renderPlot({ggplot(data = dframeA,aes(color=gender))+
    geom_line(data=males,aes_string(x='time',y=input$time))+
    geom_line(data=females,aes_string(x='time',y=input$time))+
    geom_line(data=unknowns,aes_string(x='time',y=input$time))+
    geom_line(data=unisexs,aes_string(x='time',y=input$time))})
  output$plt4=renderPlot({ggplot(data= dframe1A,aes(color=gender))+
    geom_line(data=males1,aes_string(x='time',y=input$time))+
    geom_line(data=females1,aes_string(x='time',y=input$time))+
    geom_line(data=unknowns1,aes_string(x='time',y=input$time))+
    geom_line(data=unisexs1,aes_string(x='time',y=input$time))})
}

shinyApp(ui = ui,server = server)

#------------------end of code-------------------